In [24]:
import requests
import os
import json
import time

1. Add PDF via File Upload

In [29]:
def record_pdf_source(source_id):
    """Record the pdf source id in the json file."""
    output_dir = "../settings"
    listdir = os.listdir(output_dir)

    if "pdf_source.json" in listdir:
        fr = open(os.path.join(output_dir, "pdf_source.json"))  # a represents append
        model = json.load(fr)
        fr.close()

        for i in source_id:
            model[i] = source_id[i]
        jsObj = json.dumps(model)

        with open(
            os.path.join(output_dir, "pdf_source.json"), "w"
        ) as fw:  # w represents override
            fw.write(jsObj)


# 数据准备
with open(os.path.join(os.path.abspath("."), "..", "settings", "API_KEYS.json")) as f:
    api_key = json.load(f)["ChatPDF_API"]

# ------------------------上传 pdf 文件-------------------------------------
path_to_pdf = "D://4A\！美赛\ICM//2021\D//2106028.pdf"
# path_to_pdf = input()

headers = {"x-api-key": api_key}
files = [("file", ("file", open(path_to_pdf, "rb"), "application/octet-stream"))]

response = requests.post(
    "https://api.chatpdf.com/v1/sources/add-file", headers=headers, files=files
)

if response.status_code == 200:
    print("Source ID:", response.json()["sourceId"])
    # store the source id in the json file
    source_id = {response.json()["sourceId"]: path_to_pdf}
    record_pdf_source(source_id)

    # create a history file to store the chatting history
    output_dir = "../history"
    listdir = os.listdir(output_dir)
    if str(response.json()["sourceId"]) + ".json" not in listdir:
        with open(
            os.path.join(output_dir, str(response.json()["sourceId"]) + ".json"), "w"
        ) as f:
            f.write("{}")
else:
    print("Status:", response.status_code)
    print("Error:", response.text)

Source ID: src_sLtxyrqYIpnJBIMz2TUX7


2. Chat with PDF

In [30]:
with open(os.path.join(os.path.abspath("."), "..", "settings", "API_KEYS.json")) as f:
    api_key = json.load(f)["ChatPDF_API"]
sourceId = "src_sLtxyrqYIpnJBIMz2TUX7"

headers = {
    "x-api-key": api_key,
    "Content-Type": "application/json",
}

message = {
    "role": "user",
    "content": "请用中文回答：这篇文章的建模的思路是怎么样的？又是如何检验模型的？",
}

data = {
    "sourceId": sourceId,
    "messages": [message],
}

response = requests.post(
    "https://api.chatpdf.com/v1/chats/message", headers=headers, json=data
)

if response.status_code == 200:
    print("Result:", response.json()["content"])

    # store the history of chatting
    output_dir = "../history"
    listdir = os.listdir(output_dir)
    if str(sourceId) + ".json" in listdir:
        with open(os.path.join(output_dir, str(sourceId) + ".json"), "r") as f:
            model = json.load(f)

        temp_msg = {}
        temp_msg["user"] = message["content"]
        temp_msg["assistant"] = response.json()["content"]

        current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

        model[current_time] = temp_msg
        jsObj = json.dumps(model)

        with open(
            os.path.join(output_dir, str(sourceId) + ".json"), "w"
        ) as fw:  # w: override
            fw.write(jsObj)
else:
    print("Status:", response.status_code)
    print("Error:", response.text)

Result: 这篇文章的建模思路主要是基于社交网络分析和统计方法，通过构建音乐影响网络、分析音乐特征、研究音乐流派的关系和发展、探索音乐革命的信号和先驱者、以及研究动态影响者等方面，来探究音乐对社会的影响。文章还提到了一些模型的优点和缺点，如数据不足、间接音乐影响的简化等问题。文章通过实验和数据分析来检验模型的有效性和可靠性。
